# Check Geocoding

**PART1.-** Check input_data, if the input to geocode is in the street <br />
**PART2.-** Check results through geocoding

In [31]:
import pandas as pd
import arcpy
from arcgis import GIS
import numpy as np

In [32]:
gdb = 'C:/Users/Nisleida Morales/OneDrive/Documentos/ArcGIS/Projects/Default.gdb'
streets = gdb + '/Streets'
tableToGeocode = gdb + '/tableToGeocode'

## Part 1

### Read Input Data (Streets and TableToGeocode)

In [47]:
fields_streets = ["LOC_CALLE","DESDEI","DESDED","HASTAI","HASTAD","OBJECTID"]
fields_tableToGeocode = ["LOC_CALLE","PUERTA","VALIDACION","OBJECTID"]
df_streets = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(in_table=streets,field_names = fields_streets))
df_tableToGeocode = pd.DataFrame(arcpy.da.FeatureClassToNumPyArray(in_table=tableToGeocode, field_names = fields_tableToGeocode))

### Substract records where City and Street exist

In [48]:
messageValidCityStreet = 'ValidCityStreet'
messageNotValidCityStreet = 'NotValidCityStreet'

df_tableToGeocode['VALIDACION'] = np.where(df_tableToGeocode['LOC_CALLE'].isin(df_streets['LOC_CALLE']),messageValidCityStreet,messageNotValidCityStreet)

In [49]:
df_tableToGeocode.groupby(['VALIDACION']).count()

,LOC_CALLE,PUERTA,OBJECTID
VALIDACION,,,
NotValidCityStreet,35435,35435,35435
ValidCityStreet,785115,785115,785115


In [52]:
df_valid=df_tableToGeocode[df_tableToGeocode.VALIDACION==messageValidCityStreet]

### Calculate min, max range for dual streets

In [53]:
df_streets['max_value'] = df_streets[['DESDEI','DESDED','HASTAI','HASTAD']].max(axis=1)
df_streets['min_value'] = df_streets[['DESDEI','DESDED','HASTAI','HASTAD']].min(axis=1)
df_streets

,LOC_CALLE,DESDEI,DESDED,HASTAI,HASTAD,OBJECTID,max_value,min_value
0,68-33,577,576,589,588,1,589,576
1,68-33,501,500,575,574,2,575,500
2,68-33,401,400,499,498,3,499,400
3,68-33,301,300,399,398,4,399,300
4,68-48,200,201,298,299,5,299,200
...,...,...,...,...,...,...,...,...
107589,12-R57,0,800,0,898,107590,898,0
107590,12-F85,151,0,199,0,107591,199,0
107591,12-F85,0,150,0,198,107592,198,0
107592,12-F85,0,0,0,0,107593,0,0


### Check if the input data is in the street file

In [42]:
for row_valid in df_valid.index:
    for row in df_streets.index:
        DESDE=df_streets['min_value'][row]
        HASTA=df_streets['max_value'][row]
        IS_LOC_CALLE_VALID= df_valid['LOC_CALLE'][row_valid]==df_streets['LOC_CALLE'][row]
        PUERTA=df_valid['PUERTA'][row_valid]
        
        
        if IS_LOC_CALLE_VALID and ((PUERTA>=DESDE and PUERTA<=HASTA)):
            df_valid['VALIDACIONES']= 'VALIDO'
            
print(df_valid)           


Empty DataFrame
Columns: [LOC_CALLE, PUERTA, VALIDACION, OBJECTID]
Index: []


### For original values dual range

In [ ]:
for row_valid in df_valid.index:
    for row in df_streets.index:
        DESDEI=df_streets['DESDEI'][row]
        HASTAI=df_streets['HASTAI'][row]
        DESDED=df_streets['DESDED'][row]
        HASTAD=df_streets['HASTAD'][row]
        IS_LOC_CALLE_VALID= df_valid['LOC_CALLE'][row_valid]==df_streets['LOC_CALLE'][row]
        PUERTA=df_valid['PUERTA'][row_valid]
        
        
        if IS_LOC_CALLE_VALID and ((PUERTA>=DESDEI and PUERTA<=HASTAI) or (PUERTA>=DESDED and PUERTA<=HASTAD)):
            df_valid['VALIDACIONES']= 'VALIDO'
            
print(df_valid)
            
